<a href="https://colab.research.google.com/github/AnastasiaDv491/AA-course-assignment2/blob/main/AA_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/data/archive.zip


In [ ]:
# Import libraries
import os  # For interacting with the file system
import shutil  # For managing files and directories in a cross-platform manner
import keras  # For building deep learning models
import numpy as np  # For numerical operations on arrays
from glob import glob  # For finding file paths
from tqdm import tqdm  # For progress bars

# Data preprocessing
from keras.preprocessing.image import ImageDataGenerator  # For image data augmentation

# Data visualization
import seaborn as sns  # For statistical visualizations
import plotly.graph_objs as go  # For interactive visualizations
import matplotlib.pyplot as plt  # For creating static plots

# Model architecture
from keras import Sequential  # For building sequential models
from keras.models import load_model  # For loading pre-trained models
from keras.layers import Dense, GlobalAvgPool2D as GAP, Dropout  # For defining model layers

# Training callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping  # For training callbacks

# Pre-trained models
from tensorflow.keras.applications import InceptionV3, Xception, ResNet152V2  # For using pre-trained models

In [ ]:
data_path = 'raw-img'
class_names = sorted(os.listdir(data_path))
class_names
num_classes = len(class_names)
num_classes

10

In [ ]:
class_sizes = []
for name in class_names:
    class_size = len(os.listdir(data_path + "/" + name))
    class_sizes.append(class_size)

class_name_size = dict(zip(class_names, class_sizes))

In [ ]:
sampled_data_path = './sampled-data'
if not os.path.exists(sampled_data_path):
    os.mkdir(sampled_data_path)

sample_percent = 0.1
class_names_dict = {
    'cane': 'dog',
    'cavallo': 'horse',
    'elefante': 'elephant',
    'farfalla': 'butterfly',
    'gallina': 'chicken',
    'gatto': 'cat',
    'mucca': 'cow',
    'pecora': 'sheep',
    'ragno': 'spider',
    'scoiattolo': 'squirrel'
}
for class_name in os.listdir(data_path):
    # Get the path to the original class directory
    class_path = os.path.join(data_path, class_name)
    # Get the English name of the class
    class_name_en = class_names_dict[class_name]
    # Get the path to the sampled class directory
    sampled_class_path = os.path.join(sampled_data_path, class_name_en)
    # Create the sampled class directory and fill it up only if it doesn't exist ********
    if not os.path.exists(sampled_class_path):
        os.mkdir(sampled_class_path)
        # Get a list of all the image files in the class directory
        image_files = os.listdir(class_path)
        # Calculate the number of images to sample **************
        image_class_size = class_name_size[class_name]
        if image_class_size > 2000:
            num_images = 2000
        else:
            num_images = int(image_class_size)
        # Sample the images
        sampled_images = np.random.choice(image_files, size=num_images, replace=False)
        # Copy the sampled images to the sampled class directory
        for image_name in sampled_images:
            src_path = os.path.join(class_path, image_name)
            dst_path = os.path.join(sampled_class_path, image_name)
            shutil.copyfile(src_path, dst_path)

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True,
    # rotation_range = 20,
    validation_split=0.2
)

In [ ]:
train_data = data_generator.flow_from_directory(
    sampled_data_path,
    target_size =(256,256),
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
    subset = 'training'
)
valid_data = data_generator.flow_from_directory(
    sampled_data_path, 
    target_size=(256,256), 
    class_mode='binary', 
    batch_size=32, 
    shuffle=True, 
    subset='validation')

Found 14931 images belonging to 10 classes.
Found 3731 images belonging to 10 classes.


In [ ]:
def show_image(image, image_title = None):
    plt.imshow(image)
    plt.title(image_title)
    plt.axis('off')

def get_random_data(data_tuple):
    images, labels = data_tuple
    idx = np.random.randint(len(images))
    image, label = images[idx], labels[idx]
    return image, label

plt.figure(figsize =(20,20))

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

In [ ]:
name = "ResNet152V2"
base_model = ResNet152V2(include_top=False, input_shape=(256,256,3), weights='imagenet')
base_model.trainable = False

resnet = Sequential([
    base_model,
    GAP(),
    Dense(256, activation = "relu"),
    Dropout(0.2),
    Dense(10, activation = "softmax")
], name = name)

resnet.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint(name + ".h5", save_best_only=True)
]

resnet.fit(
    train_data, validation_data=valid_data,
    epochs=10, callbacks=cbs
)



Epoch 1/10
467/467 [==============================] - 188s 382ms/step - loss: 0.4155 - accuracy: 0.8683 - val_loss: 0.2854 - val_accuracy: 0.9011
Epoch 2/10
467/467 [==============================] - 164s 351ms/step - loss: 0.2645 - accuracy: 0.9155 - val_loss: 0.2413 - val_accuracy: 0.9188
Epoch 3/10
467/467 [==============================] - 163s 347ms/step - loss: 0.2235 - accuracy: 0.9283 - val_loss: 0.2450 - val_accuracy: 0.9244
Epoch 4/10
467/467 [==============================] - 164s 351ms/step - loss: 0.2000 - accuracy: 0.9368 - val_loss: 0.2263 - val_accuracy: 0.9295
Epoch 5/10
467/467 [==============================] - 165s 354ms/step - loss: 0.1834 - accuracy: 0.9403 - val_loss: 0.2152 - val_accuracy: 0.9317
Epoch 6/10
467/467 [==============================] - 163s 348ms/step - loss: 0.1622 - accuracy: 0.9460 - val_loss: 0.2533 - val_accuracy: 0.9174
Epoch 7/10
467/467 [==============================] - 162s 347ms/step - loss: 0.1437 - accuracy: 0.9520 - val_loss: 0.2339 -

In [ ]:
model_file_path = 'ResNet152V2.h5'
loaded_model = load_model(model_file_path)
loaded_model.summary()

loaded_model.evaluate(valid_data)

OSError: ignored